In [ ]:
######### Finger count
import cv2
import time
import sys
import numpy as np
sys.path.append("../modules/")
import input_output
import processor

def track_nothing(val):
    pass

def on_gamma_trackbar(val):
    det.processors ["proc"] ["gamma correction"].set_gamma (float (val) / 100)
    
def applyBitwiseNot (img):
    res = cv2.bitwise_not (img)
    return res

def applyConvex (img):
    contours, _ = cv2.findContours(img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    hull = []
    for i in range(len(contours)):
        hull.append(cv2.convexHull(contours[i]))    
    
    for i in range(len(contours)):
#        color_contours = (0, 255, 0) # green - color for contours
        color = (255, 0, 255) # blue - color for convex hull
        # draw ith contour
        img = np.zeros((480, 640, 3), np.uint8)        
        cv2.drawContours(img, contours, i, color, 10, cv2.FILLED)
        # draw ith convex hull object
        cv2.drawContours(img, hull, i, color, 3, cv2.FILLED)    
#        defects = cv2.convexityDefects(contours[i], hull[i])
#        print (defects.shape)
#        for i in range(defects.shape[0]):
#            s,e,f,d = defects[i,0]        
#        cv2.putText (img, str (len (contours[0])), (30,30), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (255,255,255), 2)            
    return img

det = processor.Processors ()
det.add_processor ("proc")

det.add_filter (processor.gamma_correction (1), "proc", "gamma correction")
det.add_filter (processor.colorspace_to_colorspace("RGB", "HSV"), "proc", "rgb2hsv")
det.add_filter (processor.inrange ((0,0,0), (104,255,255)), "proc", "inrange")
det.add_filter (processor.GaussianBlur ("blur"), "proc", "blur")

det.add_filter (processor.morphology ("open", 0), "proc", "morph_open")
det.add_filter (processor.morphology ("close", 0), "proc", "morph_close")
det.add_filter (processor.morphology ("erode", 0), "proc", "morph_erode")
det.add_filter (processor.morphology ("dilate", 0), "proc", "morph_dilate")

det.add_filter (processor.custom_operation (applyBitwiseNot, "BitwiseNot"), "proc", "applyBitwiseNot")
#det.add_filter (processor.filter_connected_components (), "proc", "conn")
det.add_filter (processor.custom_operation (applyConvex, "convex"), "proc", "applyConvex")

cv2.namedWindow ('trackbars', cv2.WINDOW_NORMAL)
cv2.createTrackbar ("gamma", "trackbars", 100, 200, on_gamma_trackbar)
cv2.createTrackbar ("thr_l_h", "trackbars", 0, 255, track_nothing)
cv2.createTrackbar ("thr_l_s", "trackbars", 0, 255, track_nothing)
cv2.createTrackbar ("thr_l_v", "trackbars", 0, 255, track_nothing)
cv2.createTrackbar ("thr_h_h", "trackbars", 90, 255, track_nothing)
cv2.createTrackbar ("thr_h_s", "trackbars", 255, 255, track_nothing)
cv2.createTrackbar ("thr_h_v", "trackbars", 255, 255, track_nothing)

cv2.createTrackbar ("morph_op", "trackbars", 2, 20, track_nothing)
cv2.createTrackbar ("morph_cl", "trackbars", 15, 20, track_nothing)
cv2.createTrackbar ("morph_er", "trackbars", 5, 20, track_nothing)
cv2.createTrackbar ("morph_di", "trackbars", 1, 10, track_nothing)
cv2.createTrackbar ("blur", "trackbars", 4, 20, track_nothing)

#cv2.createTrackbar ("area_l", "trackbars", 15000, 300000, track_nothing)
#cv2.createTrackbar ("area_h", "trackbars", 120000, 300000, track_nothing)

print ('Start init of the front camera')
start = time.time()
try:
    source  = input_output.Source ("1")
except:
    print ("Cam not found")
    source.release()    
end = time.time()
print ('End of front camera init. Duration {sec} seconds.'.format(sec = end - start))

while (True): 
    det.processors ["proc"] ["inrange"].set_ths (
        (cv2.getTrackbarPos ("thr_l_h", "trackbars"),
         cv2.getTrackbarPos ("thr_l_s", "trackbars"),
         cv2.getTrackbarPos ("thr_l_v", "trackbars")),
        (cv2.getTrackbarPos ("thr_h_h", "trackbars"),
         cv2.getTrackbarPos ("thr_h_s", "trackbars"),
         cv2.getTrackbarPos ("thr_h_v", "trackbars")))    

    det.processors ["proc"] ["morph_open"].ker_sz = cv2.getTrackbarPos ("morph_op", "trackbars")
    det.processors ["proc"] ["morph_close"].ker_sz = cv2.getTrackbarPos ("morph_cl", "trackbars")
    det.processors ["proc"] ["morph_erode"].ker_sz = cv2.getTrackbarPos ("morph_er", "trackbars")    
    det.processors ["proc"] ["morph_dilate"].ker_sz = cv2.getTrackbarPos ("morph_di", "trackbars")
    
    ksize_blur = cv2.getTrackbarPos ("blur", "trackbars")
    if ksize_blur % 2 == 0:
        ksize_blur = ksize_blur + 1
    det.processors ["proc"] ["blur"].ker_sz =  ksize_blur     
    
#    det.processors ["proc"] ["conn"].area_low = cv2.getTrackbarPos ("area_l", "trackbars")  
#    det.processors ["proc"] ["conn"].area_high = cv2.getTrackbarPos ("area_h", "trackbars")      
    
    frame = source.get_frame ()
    _,_ = det.process (frame, "proc")
  
    stages_picts = det.get_stages_picts ("proc")
    
    cv2.imshow ("Counter", input_output.form_grid(stages_picts, window_x_sz=1000))

    key = cv2.waitKey (1)
    if key == 27:  # wait for ESC key to ex it
         break
 

print ('low hue = {}'.format (cv2.getTrackbarPos ("thr_l_h", "trackbars")))
print ('low sat = {}'.format (cv2.getTrackbarPos ("thr_l_s", "trackbars")))
print ('low val = {}'.format (cv2.getTrackbarPos ("thr_l_v", "trackbars")))
print ('high hue = {}'.format (cv2.getTrackbarPos ("thr_h_h", "trackbars")))
print ('high sat = {}'.format (cv2.getTrackbarPos ("thr_h_s", "trackbars")))
print ('high val = {}'.format (cv2.getTrackbarPos ("thr_h_v", "trackbars")))

print ('morph_op = {}'.format (cv2.getTrackbarPos ("morph_op", "trackbars")))
print ('morph_cl = {}'.format (cv2.getTrackbarPos ("morph_cl", "trackbars")))
print ('morph_er = {}'.format (cv2.getTrackbarPos ("morph_er", "trackbars")))
print ('morph_di = {}'.format (cv2.getTrackbarPos ("morph_di", "trackbars")))
print ('blur = {}'.format (cv2.getTrackbarPos ("blur", "trackbars")))


source.release()
cv2.destroyAllWindows()        

Start init of the front camera
End of front camera init. Duration 4.064694166183472 seconds.
